In [3]:
import os
import pandas as pd
import geopandas as gpd
import asyncio

from shapely import wkt
from ipyleaflet import Map, GeoData, LayersControl, basemaps
from IPython.display import display

mutex = asyncio.Lock()

FILE_PATH = 'data/grids/main_roads_grids.csv'

def load_geodata(file_path):
    gdf = gpd.read_file(file_path)
    gdf['geometry'] = gdf['wkt'].apply(lambda x: wkt.loads(x))
    gdf.set_geometry('geometry', inplace=True)
    gdf.set_crs(epsg=4326, inplace=True)
    return gdf

def create_map(df):
    m = Map(center=(df['lat'].mean(), df['lon'].mean()), scroll_wheel_zoom=True, zoom=11, basemap=basemaps.OpenStreetMap.Mapnik)
    m.layout.height = '1000px'
    return m

def save_selected_grids():
    if selected_grids_gdf.empty:
        if os.path.exists(FILE_PATH):
            os.remove(FILE_PATH)
    else: 
        selected_grids_gdf['id'] = selected_grids_gdf.index
        selected_grids_gdf.rename_geometry('wkt').to_csv(FILE_PATH, index=False)

def handle_click_all_grids_layer(**kwargs):
    asyncio.create_task(_handle_click_all_grids_layer(**kwargs))

async def _handle_click_all_grids_layer(**kwargs):
    async with mutex:
        properties = kwargs.get('feature').get('properties')
        selected_grids_gdf.loc[properties['grelha_id']] = {'geometry': wkt.loads(properties['wkt'])}
        selected_grids_gdf.set_geometry('geometry', inplace=True)
        selected_grids_gdf.set_crs(epsg=4326, inplace=True)
        selected_grids_layer.geo_dataframe = selected_grids_gdf
        save_selected_grids()
    
    await asyncio.sleep(0.1)

def handle_click_selected_grids_layer(**kwargs):
    asyncio.create_task(_handle_click_selected_grids_layer(**kwargs))

async def _handle_click_selected_grids_layer(**kwargs):
    async with mutex:
        id = kwargs.get('feature')['id']
        selected_grids_gdf.drop(id, inplace=True)
        selected_grids_layer.geo_dataframe = selected_grids_gdf
        save_selected_grids()

    await asyncio.sleep(0.1)

# Load your GeoDataFrame
all_grids_gdf = load_geodata('data/wktComplete.csv')

if os.path.exists(FILE_PATH):
    selected_grids_gdf = load_geodata(FILE_PATH)
    selected_grids_gdf.set_index('id', inplace=True)
    selected_grids_gdf.drop('wkt', axis=1, inplace=True)
else:
    selected_grids_gdf = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry', crs='EPSG:4326')

# Convert the GeoDataFrame to a DataFrame
df = pd.DataFrame(all_grids_gdf)

# Calculate the centroid of each polygon
df['centroid'] = all_grids_gdf['geometry'].centroid

# Extract the latitude and longitude from the centroid column
df['lon'] = df['centroid'].apply(lambda point: point.x)
df['lat'] = df['centroid'].apply(lambda point: point.y)

# Create a map
m = create_map(df)

all_grids_gdf = all_grids_gdf.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

all_grids_layer = GeoData(geo_dataframe=all_grids_gdf, style={'color': 'blue', 'opacity':1, 'weight':1, 'fillOpacity':0.1}, name='AllGrids')
selected_grids_layer = GeoData(geo_dataframe=selected_grids_gdf, style={'color': 'red', 'opacity':1, 'weight':1, 'fillOpacity':0.1}, name='SelectedGrids')

all_grids_layer.on_click(handle_click_all_grids_layer)
selected_grids_layer.on_click(handle_click_selected_grids_layer)

m.add_layer(all_grids_layer)
m.add_layer(selected_grids_layer)
m.add_control(LayersControl())

display(m)

/tmp/ipykernel_16934/48223492.py:74: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['centroid'] = all_grids_gdf['geometry'].centroid


Map(center=[38.74256107238327, -9.156505551146509], controls=(ZoomControl(options=['position', 'zoom_in_text',…